In [1]:


#imports
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from scipy.stats import kurtosis, skew # it's to explore some statistics of numerical values

import os
import json
from pandas import json_normalize

def load_df(csv_path='../input/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df


In [2]:
train = load_df(nrows=100000, csv_path="data/train.csv")
test = load_df("data/test.csv", 100000)
pd.set_option('display.max_columns', None)

Loaded train.csv. Shape: (100000, 55)
Loaded test.csv. Shape: (100000, 53)


In [3]:
def missing_values(data):
    total = data.isnull().sum().sort_values(ascending = False) # getting the sum of null values and ordering
    percent = (data.isnull().sum() / data.isnull().count() * 100 ).sort_values(ascending = False) #getting the percent and order of null
    df = pd.concat([total, percent], axis=1, keys=['Total', 'Percent']) # Concatenating the total and percent
    print("Total columns at least one Values: ")
    print (df[~(df['Total'] == 0)]) # Returning values of nulls different of 0
    
    print("\n Total of Sales % of Total: ", round((train[train['totals.transactionRevenue'] != np.nan]['totals.transactionRevenue'].count() / len(train['totals.transactionRevenue']) * 100),4))
    
    return 

In [4]:
missing_values(train)

Total columns at least one Values: 
                                              Total  Percent
trafficSource.campaignCode                    99999   99.999
trafficSource.adContent                       98675   98.675
totals.transactionRevenue                     98601   98.601
trafficSource.adwordsClickInfo.isVideoAd      97426   97.426
trafficSource.adwordsClickInfo.adNetworkType  97426   97.426
trafficSource.adwordsClickInfo.slot           97426   97.426
trafficSource.adwordsClickInfo.page           97426   97.426
trafficSource.adwordsClickInfo.gclId          97375   97.375
trafficSource.isTrueDirect                    69546   69.546
trafficSource.referralPath                    63527   63.527
trafficSource.keyword                         55782   55.782
totals.bounces                                51084   51.084
totals.newVisits                              22737   22.737
totals.pageviews                                  7    0.007

 Total of Sales % of Total:  1.399


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 55 columns):
 #   Column                                             Non-Null Count   Dtype 
---  ------                                             --------------   ----- 
 0   channelGrouping                                    100000 non-null  object
 1   date                                               100000 non-null  int64 
 2   fullVisitorId                                      100000 non-null  object
 3   sessionId                                          100000 non-null  object
 4   socialEngagementType                               100000 non-null  object
 5   visitId                                            100000 non-null  int64 
 6   visitNumber                                        100000 non-null  int64 
 7   visitStartTime                                     100000 non-null  int64 
 8   device.browser                                     100000 non-null  object
 9   devic

In [6]:
# Find the columns where values are not available
useless_cols = [col for col in train.columns if train[col].eq("not available in demo dataset").all()]
# Drop these columns from the dataframe
train.drop(useless_cols,
        axis=1,
        inplace=True)

# Find the columns where values are not available
useless_cols = [col for col in train.columns if train[col].isna().all()]
# Drop these columns from the dataframe
train.drop(useless_cols,
        axis=1,
        inplace=True)

C:\ProgramData\Anaconda\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [7]:
const_cols = [c for c in train.columns if train[c].nunique(dropna=False)==1 ]
const_cols

['socialEngagementType', 'totals.visits']

In [8]:
print("Variables not in test but in train : ", set(train.columns).difference(set(test.columns)))

Variables not in test but in train :  {'trafficSource.campaignCode', 'totals.transactionRevenue'}


In [9]:
cols_to_drop = const_cols + ['sessionId']

train_df = train.drop(cols_to_drop + ["trafficSource.campaignCode"], axis=1)
test_df = test.drop(cols_to_drop, axis=1)

In [10]:
train_df.shape

(100000, 34)

In [11]:
from sklearn import model_selection, preprocessing, metrics
import datetime

# Impute 0 for missing target values
train_df["totals.transactionRevenue"].fillna(0.0, inplace=True)
train_df["totals.pageviews"].fillna(0.0, inplace=True)
train_df["totals.bounces"].fillna(0.0, inplace=True)
train_df["totals.newVisits"].fillna(0.0, inplace=True)
train_y = train_df["totals.transactionRevenue"].values
train_id = train_df["fullVisitorId"].values
test_id = test_df["fullVisitorId"].values


# label encode the categorical variables and convert the numerical variables to float
cat_cols = ["channelGrouping", "device.browser", 
            "device.deviceCategory", "device.operatingSystem", 
            "geoNetwork.city", "geoNetwork.continent", 
            "geoNetwork.country", "geoNetwork.metro",
            "geoNetwork.networkDomain", "geoNetwork.region", 
            "geoNetwork.subContinent", "trafficSource.adContent", 
            "trafficSource.adwordsClickInfo.adNetworkType", 
            "trafficSource.adwordsClickInfo.gclId", 
            "trafficSource.adwordsClickInfo.page", 
            "trafficSource.adwordsClickInfo.slot", "trafficSource.campaign",
            "trafficSource.keyword", "trafficSource.medium", 
            "trafficSource.referralPath", "trafficSource.source",
            'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.isTrueDirect']
for col in cat_cols:
    print(col)
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(train_df[col].values.astype('str')) + list(test_df[col].values.astype('str')))
    train_df[col] = lbl.transform(list(train_df[col].values.astype('str')))
    test_df[col] = lbl.transform(list(test_df[col].values.astype('str')))

train_df["totals.transactionRevenue"] = train_df["totals.transactionRevenue"].astype(float)

num_cols = ["totals.hits", "totals.pageviews", "visitNumber", "visitStartTime", 'totals.bounces',  'totals.newVisits']    
for col in num_cols:
    train_df[col] = train_df[col].astype(float)
    test_df[col] = test_df[col].astype(float)

# Split the train dataset into development and valid based on time 
train_df["totals.transactionRevenue"] = train_df["totals.transactionRevenue"].astype('float')
dev_df = train_df[train_df['date']<=20170531]
val_df = train_df[train_df['date']>2017531]
dev_y = np.log1p(dev_df["totals.transactionRevenue"].values)
val_y = np.log1p(val_df["totals.transactionRevenue"].values)

dev_X = dev_df[cat_cols + num_cols] 
val_X = val_df[cat_cols + num_cols] 
test_X = test_df[cat_cols + num_cols]

channelGrouping
device.browser
device.deviceCategory
device.operatingSystem
geoNetwork.city
geoNetwork.continent
geoNetwork.country
geoNetwork.metro
geoNetwork.networkDomain
geoNetwork.region
geoNetwork.subContinent
trafficSource.adContent
trafficSource.adwordsClickInfo.adNetworkType
trafficSource.adwordsClickInfo.gclId
trafficSource.adwordsClickInfo.page
trafficSource.adwordsClickInfo.slot
trafficSource.campaign
trafficSource.keyword
trafficSource.medium
trafficSource.referralPath
trafficSource.source
trafficSource.adwordsClickInfo.isVideoAd
trafficSource.isTrueDirect


In [12]:
train_df.shape

(100000, 34)

#### Find kurtosis and Skewness of Transaction Revenue

In [13]:
print('Excess kurtosis of normal distribution (should be 0): {}'.format(
    kurtosis(train_df[train_df["totals.transactionRevenue"] > 0]["totals.transactionRevenue"])))
print( 'Skewness of normal distribution (should be 0): {}'.format(
    skew((train_df[train_df["totals.transactionRevenue"] > 0]["totals.transactionRevenue"]))))

Excess kurtosis of normal distribution (should be 0): 127.13432610754907
Skewness of normal distribution (should be 0): 9.277714428055667


In [14]:
import lightgbm as lgb

# custom function to run light gbm model
def run_lgb(train_X, train_y, val_X, val_y, test_X):
    params = {
        "objective" : "regression",
        "metric" : "rmse", 
        "num_leaves" : 30,
        "min_child_samples" : 100,
        "learning_rate" : 0.1,
        "bagging_fraction" : 0.7,
        "feature_fraction" : 0.5,
        "bagging_frequency" : 5,
        "bagging_seed" : 2018,
        "verbosity" : -1
    }
    
    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    model = lgb.train(params, lgtrain, 1000, valid_sets=[lgval], early_stopping_rounds=100, verbose_eval=100)
    
    pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    pred_val_y = model.predict(val_X, num_iteration=model.best_iteration)
    return pred_test_y, model, pred_val_y

# Training the model #
pred_test, model, pred_val = run_lgb(dev_X, dev_y, val_X, val_y, test_X)

[LightGBM] [Warning] Unknown parameter: bagging_frequency
Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 1.57137
[200]	valid_0's rmse: 1.52719
[300]	valid_0's rmse: 1.49869
[400]	valid_0's rmse: 1.47154
[500]	valid_0's rmse: 1.44936
[600]	valid_0's rmse: 1.4302
[700]	valid_0's rmse: 1.4098
[800]	valid_0's rmse: 1.3924
[900]	valid_0's rmse: 1.37487
[1000]	valid_0's rmse: 1.36078
Did not meet early stopping. Best iteration is:
[1000]	valid_0's rmse: 1.36078


In [15]:
from sklearn import metrics
pred_val[pred_val<0] = 0
val_pred_df = pd.DataFrame({"fullVisitorId":val_df["fullVisitorId"].values})
val_pred_df["transactionRevenue"] = val_df["totals.transactionRevenue"].values
val_pred_df["PredictedRevenue"] = np.expm1(pred_val)
#print(np.sqrt(metrics.mean_squared_error(np.log1p(val_pred_df["transactionRevenue"].values), np.log1p(val_pred_df["PredictedRevenue"].values))))
val_pred_df = val_pred_df.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()
print(np.sqrt(metrics.mean_squared_error(np.log1p(val_pred_df["transactionRevenue"].values), np.log1p(val_pred_df["PredictedRevenue"].values))))

1.3749812233864607


<ipython-input-15-2a729c880d81>:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  val_pred_df = val_pred_df.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()


In [16]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    dev_X, dev_y, random_state=0)

In [17]:
X_train.head()

,channelGrouping,device.browser,device.deviceCategory,device.operatingSystem,geoNetwork.city,geoNetwork.continent,geoNetwork.country,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.region,geoNetwork.subContinent,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.isTrueDirect,totals.hits,totals.pageviews,visitNumber,visitStartTime,totals.bounces,totals.newVisits
85573,1,12,0,17,459,2,190,78,7450,318,12,47,3,8098,6,3,10,626,2,1386,16,1,1,1.0,1.0,1.0,1.480437e+09,1.0,1.0
35822,2,12,0,17,317,4,58,0,7970,124,22,47,3,8098,6,3,3,626,0,1386,0,1,0,2.0,2.0,1.0,1.493707e+09,0.0,1.0
80607,2,12,1,1,0,2,142,0,3206,0,15,47,3,8098,6,3,3,626,0,1386,0,1,0,5.0,5.0,1.0,1.471878e+09,0.0,1.0
875,4,12,0,3,459,2,190,78,0,318,12,47,3,8098,6,3,3,5,5,1386,74,1,1,18.0,12.0,1.0,1.472838e+09,0.0,1.0
87560,2,12,1,1,0,2,190,0,4047,0,12,47,3,8098,6,3,3,626,0,1386,0,1,0,1.0,1.0,1.0,1.489842e+09,1.0,1.0


In [18]:
reg = GradientBoostingRegressor(random_state=0)
reg.fit(X_train, y_train)
val_pred = reg.predict(X_test[1:2])

In [19]:
reg.score(X_test, y_test)

0.3315152747907183

We can tell that there appears to be no missing values

In [20]:
payingCustomers = train_df.loc[train_df['totals.transactionRevenue'].notna()]
payingCustomers

,channelGrouping,date,fullVisitorId,visitId,visitNumber,visitStartTime,device.browser,device.operatingSystem,device.isMobile,device.deviceCategory,geoNetwork.continent,geoNetwork.subContinent,geoNetwork.country,geoNetwork.region,geoNetwork.metro,geoNetwork.city,geoNetwork.networkDomain,totals.hits,totals.pageviews,totals.bounces,totals.newVisits,totals.transactionRevenue,trafficSource.campaign,trafficSource.source,trafficSource.medium,trafficSource.keyword,trafficSource.isTrueDirect,trafficSource.referralPath,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adContent
0,4,20160902,1131660440785968503,1472830385,1.0,1.472830e+09,12,17,False,0,3,21,182,129,0,189,10216,1.0,1.0,1.0,1.0,0.0,3,74,5,5,1,1386,6,3,8098,3,1,47
1,4,20160902,377306020877927890,1472880147,1.0,1.472880e+09,17,7,False,0,5,1,9,318,78,459,2805,1.0,1.0,1.0,1.0,0.0,3,74,5,5,1,1386,6,3,8098,3,1,47
2,4,20160902,3895546263509774583,1472865386,1.0,1.472865e+09,12,17,False,0,4,19,164,68,0,233,10632,1.0,1.0,1.0,1.0,0.0,3,74,5,5,1,1386,6,3,8098,3,1,47
3,4,20160902,4763447161404445595,1472881213,1.0,1.472881e+09,40,6,False,0,3,16,82,318,78,459,10632,1.0,1.0,1.0,1.0,0.0,3,74,5,285,1,1386,6,3,8098,3,1,47
4,4,20160902,27294437909732085,1472822600,2.0,1.472823e+09,12,1,True,1,4,13,189,318,78,459,10632,1.0,1.0,1.0,0.0,0.0,3,74,5,5,0,1386,6,3,8098,3,1,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,7,20161030,06201955442296075,1477892744,1.0,1.477893e+09,34,7,False,0,3,16,178,30,0,36,10077,1.0,1.0,1.0,1.0,0.0,3,221,6,626,1,1086,6,3,8098,3,1,47
99996,7,20161030,3638854205116467643,1477840657,1.0,1.477841e+09,12,17,False,0,3,6,175,318,78,459,5350,1.0,1.0,1.0,1.0,0.0,3,221,6,626,1,1099,6,3,8098,3,1,47
99997,7,20161030,5058770420082063061,1477889694,1.0,1.477890e+09,12,17,False,0,3,16,194,318,78,459,11012,1.0,1.0,1.0,1.0,0.0,3,221,6,626,1,1094,6,3,8098,3,1,47
99998,7,20161030,5749057926198995016,1477842029,1.0,1.477842e+09,34,7,False,0,1,11,3,318,78,459,10632,1.0,1.0,1.0,1.0,0.0,3,221,6,626,1,1028,6,3,8098,3,1,47


In [21]:
payingCustomers.describe()

,channelGrouping,date,visitId,visitNumber,visitStartTime,device.browser,device.operatingSystem,device.deviceCategory,geoNetwork.continent,geoNetwork.subContinent,geoNetwork.country,geoNetwork.region,geoNetwork.metro,geoNetwork.city,geoNetwork.networkDomain,totals.hits,totals.pageviews,totals.bounces,totals.newVisits,totals.transactionRevenue,trafficSource.campaign,trafficSource.source,trafficSource.medium,trafficSource.keyword,trafficSource.isTrueDirect,trafficSource.referralPath,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adContent
count,100000.000000,1.000000e+05,1.000000e+05,100000.000000,1.000000e+05,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,1.000000e+05,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,4.622340,2.016536e+07,1.484710e+09,2.274210,1.484710e+09,17.387900,11.346960,0.290950,2.707470,13.310700,143.570560,229.92268,55.450740,358.774740,5120.953290,4.622350,3.867880,0.489160,0.772630,1.819409e+06,3.263020,102.838420,4.448750,359.324910,0.695460,1132.720250,5.845720,2.973790,8012.662750,2.948530,0.974260,46.647690
std,1.769928,4.694358e+03,9.166865e+06,9.260941,9.166865e+06,9.177854,6.684055,0.520481,0.879297,4.627347,59.950538,114.94056,32.576252,142.102674,4532.296014,9.620067,6.981753,0.499885,0.419136,3.901549e+07,1.354422,77.124648,2.077161,306.376542,0.460215,438.682595,0.949403,0.162675,610.642978,0.316673,0.158359,3.303592
min,0.000000,2.016081e+07,1.470896e+09,1.000000,1.470899e+09,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000e+00,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000
25%,4.000000,2.016111e+07,1.478832e+09,1.000000,1.478832e+09,12.000000,7.000000,0.000000,2.000000,12.000000,86.000000,119.00000,21.000000,273.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000e+00,3.000000,74.000000,5.000000,5.000000,0.000000,1013.000000,6.000000,3.000000,8098.000000,3.000000,1.000000,47.000000
50%,4.000000,2.016121e+07,1.481667e+09,1.000000,1.481667e+09,12.000000,7.000000,0.000000,2.000000,12.000000,182.000000,318.00000,78.000000,459.000000,4392.000000,2.000000,2.000000,0.000000,1.000000,0.000000e+00,3.000000,74.000000,5.000000,626.000000,1.000000,1386.000000,6.000000,3.000000,8098.000000,3.000000,1.000000,47.000000
75%,6.000000,2.017050e+07,1.493684e+09,1.000000,1.493684e+09,19.000000,17.000000,1.000000,3.000000,16.000000,190.000000,318.00000,78.000000,459.000000,10216.000000,4.000000,4.000000,1.000000,1.000000,0.000000e+00,3.000000,155.000000,6.000000,626.000000,1.000000,1386.000000,6.000000,3.000000,8098.000000,3.000000,1.000000,47.000000
max,7.000000,2.017080e+07,1.501657e+09,389.000000,1.501657e+09,42.000000,20.000000,2.000000,5.000000,22.000000,197.000000,318.00000,78.000000,459.000000,11639.000000,500.000000,400.000000,1.000000,1.000000,5.498000e+09,29.000000,221.000000,6.000000,992.000000,1.000000,1386.000000,6.000000,3.000000,8098.000000,3.000000,1.000000,58.000000


In [22]:
payingCustomers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 34 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   channelGrouping                               100000 non-null  int64  
 1   date                                          100000 non-null  int64  
 2   fullVisitorId                                 100000 non-null  object 
 3   visitId                                       100000 non-null  int64  
 4   visitNumber                                   100000 non-null  float64
 5   visitStartTime                                100000 non-null  float64
 6   device.browser                                100000 non-null  int64  
 7   device.operatingSystem                        100000 non-null  int64  
 8   device.isMobile                               100000 non-null  bool   
 9   device.deviceCategory                         100

## Get the data

In [23]:
#imports
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

import os
import json
from pandas import json_normalize

json_cols = ['device', 'geoNetwork', 'totals', 'trafficSource']

def load_df(csv_path='../input/train.csv', nrows=None):
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in json_cols}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in json_cols:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

## Splitting the data

I just don't see how this works with a pipeline..

## Transformers

#### Transformer for removing unwanted features

In [24]:
from sklearn.base import TransformerMixin
from sklearn.base import BaseEstimator

class FeatureReducer(BaseEstimator, TransformerMixin):
    def __init__(self, features):
        self.features = features
    
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X.drop(self.features, axis=1)

#### Json flattening

In [25]:
def JsonFlattener(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

#### Transformers for labeling, converting and imputing the data

In [26]:
class Labeler(BaseEstimator, TransformerMixin):
    def __init__(self, cat_cols):
        self.cat_cols = cat_cols
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        for col in self.cat_cols:
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(X[col].values.astype('str')))
            X[col] = lbl.transform(list(X[col].values.astype('str')))
        return X

class Floatinator(BaseEstimator, TransformerMixin):
    def __init__(self, num_cols):
        self.num_cols = num_cols
    
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        for col in self.num_cols:
            X[col] = X[col].astype(float)
        return X
    
class SimplerImputer(BaseEstimator, TransformerMixin):
    def __init__(self, cols):
        self.cols = cols
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        for col in self.cols:
            X[col].fillna(0.0, inplace=True)
        return X


#### Splitting the data

Collect all columns which need to be dropped

In [27]:
useless_cols = [col for col in train.columns 
                if train[col].isna().all() 
                or train[col].eq("not available in demo dataset").all()
                or train[col].nunique(dropna=False)==1]
useless_cols = useless_cols + ["trafficSource.campaignCode"] + ["sessionId"]
useless_cols

C:\ProgramData\Anaconda\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


['socialEngagementType',
 'totals.visits',
 'trafficSource.campaignCode',
 'sessionId']

#### Declare columns which need to be flattened:

#### Declare categorical columns

In [28]:
categorical_cols = ["channelGrouping", "device.browser", 
            "device.deviceCategory", "device.operatingSystem", 
            "geoNetwork.city", "geoNetwork.continent", 
            "geoNetwork.country", "geoNetwork.metro",
            "geoNetwork.networkDomain", "geoNetwork.region", 
            "geoNetwork.subContinent", "trafficSource.adContent", 
            "trafficSource.adwordsClickInfo.adNetworkType", 
            "trafficSource.adwordsClickInfo.gclId", 
            "trafficSource.adwordsClickInfo.page", 
            "trafficSource.adwordsClickInfo.slot", "trafficSource.campaign",
            "trafficSource.keyword", "trafficSource.medium", 
            "trafficSource.referralPath", "trafficSource.source",
            'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.isTrueDirect']

### Declare numerical columns

In [29]:
num_cols = ["totals.hits", "totals.pageviews", "visitNumber", "visitStartTime", 'totals.bounces',  'totals.newVisits']

### Imports

In [30]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

In [31]:
for col in cat_cols:
    print(col)
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(train_df[col].values.astype('str')) + list(test_df[col].values.astype('str')))
    train_df[col] = lbl.transform(list(train_df[col].values.astype('str')))
    test_df[col] = lbl.transform(list(test_df[col].values.astype('str')))

channelGrouping
device.browser
device.deviceCategory
device.operatingSystem
geoNetwork.city
geoNetwork.continent
geoNetwork.country
geoNetwork.metro
geoNetwork.networkDomain
geoNetwork.region
geoNetwork.subContinent
trafficSource.adContent
trafficSource.adwordsClickInfo.adNetworkType
trafficSource.adwordsClickInfo.gclId
trafficSource.adwordsClickInfo.page
trafficSource.adwordsClickInfo.slot
trafficSource.campaign
trafficSource.keyword
trafficSource.medium
trafficSource.referralPath
trafficSource.source
trafficSource.adwordsClickInfo.isVideoAd
trafficSource.isTrueDirect


In [32]:
prepare_data_pipeline = Pipeline([
    #('flatten', JsonFlattener(json_cols)),
    ('reduce', FeatureReducer(useless_cols))
])

"""label_encoding_pipeline = Pipeline([
    ("encode_labels", ColumnTransformer(
        ("label_transformer", LabelEncoder(), categorical_cols)
    ))
])"""

full_pipeline = Pipeline([
    ("impute_revenue", ColumnTransformer(
        ('revenue_imputer', SimpleImputer(strategy="constant", fill_value=0), "totals.transactionRevenue")
    )),
    ('reduce', FeatureReducer(useless_cols)),
    ("encode_labels", ColumnTransformer(
        ("label_transformer", LabelEncoder(), categorical_cols)
    ))
])

"""
train_pipeline = Pipeline([
    full_pipeline,
    ("impute_revenue", ColumnTransformer([
        ('revenue_imputer', SimpleImputer(strategy="constant", fill_value=0), "totals.transactionRevenue")
    ]))
])"""

## Temp
some_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="constant", fill_value=0)),
    ('GBR', GradientBoostingRegressor(random_state=0))
])

In [33]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

some_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="constant", fill_value=0))
])

## Get data

In [34]:
train = load_df(nrows=100000, csv_path="data/train.csv")
test = load_df("data/test.csv", 100000)
pd.set_option('display.max_columns', None)

Loaded train.csv. Shape: (100000, 55)
Loaded test.csv. Shape: (100000, 53)


#### Declare columns which need to be flattened:

In [35]:
json_cols = ['device', 'geoNetwork', 'totals', 'trafficSource']

#### Declare categorical columns

In [36]:
categorical_cols = ["channelGrouping", "device.browser", 
            "device.deviceCategory", "device.operatingSystem", 
            "geoNetwork.city", "geoNetwork.continent", 
            "geoNetwork.country", "geoNetwork.metro",
            "geoNetwork.networkDomain", "geoNetwork.region", 
            "geoNetwork.subContinent", "trafficSource.adContent", 
            "trafficSource.adwordsClickInfo.adNetworkType", 
            "trafficSource.adwordsClickInfo.gclId", 
            "trafficSource.adwordsClickInfo.page", 
            "trafficSource.adwordsClickInfo.slot", "trafficSource.campaign",
            "trafficSource.keyword", "trafficSource.medium", 
            "trafficSource.referralPath", "trafficSource.source",
            'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.isTrueDirect']

### Declare numerical columns

In [37]:
num_cols = ["totals.transactionRevenue", "totals.hits", "totals.pageviews", "visitNumber", "visitStartTime", 'totals.bounces',  'totals.newVisits']

## Preprocess data for training

In [38]:
# Data is already flattened

# Remove useless columns
train = FeatureReducer(useless_cols).transform(train)

In [39]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 51 columns):
 #   Column                                             Non-Null Count   Dtype 
---  ------                                             --------------   ----- 
 0   channelGrouping                                    100000 non-null  object
 1   date                                               100000 non-null  int64 
 2   fullVisitorId                                      100000 non-null  object
 3   visitId                                            100000 non-null  int64 
 4   visitNumber                                        100000 non-null  int64 
 5   visitStartTime                                     100000 non-null  int64 
 6   device.browser                                     100000 non-null  object
 7   device.browserVersion                              100000 non-null  object
 8   device.browserSize                                 100000 non-null  object
 9   devic

In [40]:
# Impute values
cols_to_impute = [
    "totals.transactionRevenue",
    "totals.pageviews",
    "totals.bounces",
    "totals.newVisits"
]
train = SimplerImputer(cols_to_impute).transform(train)

In [41]:
train = Labeler(cat_cols).transform(train)

In [42]:
train = Floatinator(num_cols).transform(train)

In [43]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 51 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   channelGrouping                                    100000 non-null  int64  
 1   date                                               100000 non-null  int64  
 2   fullVisitorId                                      100000 non-null  object 
 3   visitId                                            100000 non-null  int64  
 4   visitNumber                                        100000 non-null  float64
 5   visitStartTime                                     100000 non-null  float64
 6   device.browser                                     100000 non-null  int64  
 7   device.browserVersion                              100000 non-null  object 
 8   device.browserSize                                 100000 non-null  object 

In [44]:
train.head()

,channelGrouping,date,fullVisitorId,visitId,visitNumber,visitStartTime,device.browser,device.browserVersion,device.browserSize,device.operatingSystem,device.operatingSystemVersion,device.isMobile,device.mobileDeviceBranding,device.mobileDeviceModel,device.mobileInputSelector,device.mobileDeviceInfo,device.mobileDeviceMarketingName,device.flashVersion,device.language,device.screenColors,device.screenResolution,device.deviceCategory,geoNetwork.continent,geoNetwork.subContinent,geoNetwork.country,geoNetwork.region,geoNetwork.metro,geoNetwork.city,geoNetwork.cityId,geoNetwork.networkDomain,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.networkLocation,totals.hits,totals.pageviews,totals.bounces,totals.newVisits,totals.transactionRevenue,trafficSource.campaign,trafficSource.source,trafficSource.medium,trafficSource.keyword,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.isTrueDirect,trafficSource.referralPath,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adContent
0,4,20160902,1131660440785968503,1472830385,1.0,1.472830e+09,5,not available in demo dataset,not available in demo dataset,12,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,0,3,21,168,93,0,118,not available in demo dataset,6360,not available in demo dataset,not available in demo dataset,not available in demo dataset,1.0,1.0,1.0,1.0,0.0,0,49,5,5,not available in demo dataset,1,527,4,2,2389,2,1,21
1,4,20160902,377306020877927890,1472880147,1.0,1.472880e+09,8,not available in demo dataset,not available in demo dataset,7,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,0,5,1,9,217,52,289,not available in demo dataset,1762,not available in demo dataset,not available in demo dataset,not available in demo dataset,1.0,1.0,1.0,1.0,0.0,0,49,5,5,not available in demo dataset,1,527,4,2,2389,2,1,21
2,4,20160902,3895546263509774583,1472865386,1.0,1.472865e+09,5,not available in demo dataset,not available in demo dataset,12,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,0,4,19,151,49,0,145,not available in demo dataset,6597,not available in demo dataset,not available in demo dataset,not available in demo dataset,1.0,1.0,1.0,1.0,0.0,0,49,5,5,not available in demo dataset,1,527,4,2,2389,2,1,21
3,4,20160902,4763447161404445595,1472881213,1.0,1.472881e+09,26,not available in demo dataset,not available in demo dataset,6,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,0,3,16,76,217,52,289,not available in demo dataset,6597,not available in demo dataset,not available in demo dataset,not available in demo dataset,1.0,1.0,1.0,1.0,0.0,0,49,5,203,not available in demo dataset,1,527,4,2,2389,2,1,21
4,4,20160902,27294437909732085,1472822600,2.0,1.472823e+09,5,not available in demo dataset,not available in demo dataset,1,not available in demo dataset,True,not available in demo dataset,not available in demo 

## Split data

### ENDRET train til train_df kun for å få kjøre

In [45]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    train_df, 
    train_df["totals.transactionRevenue"], test_size=0.4, random_state=42)

In [46]:
y_train.head()

40507    0.0
72707    0.0
90912    0.0
28532    0.0
13006    0.0
Name: totals.transactionRevenue, dtype: float64

In [47]:
y_test.describe()

count    4.000000e+04
mean     1.813006e+06
std      3.129121e+07
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      2.365500e+09
Name: totals.transactionRevenue, dtype: float64

In [48]:
X_train = X_train.drop('totals.transactionRevenue', axis=1)
gbr = GradientBoostingRegressor(random_state=0)
gbr.fit(X_train, y_train)
gbr.score(X_test.drop('totals.transactionRevenue', axis=1), y_test)

-0.7584070722584437

In [49]:
X_test = X_test.drop('totals.transactionRevenue', axis=1)
pred = gbr.predict(X_test)

In [50]:
pred = pred.clip(min=0)
pred

array([     0.        , 344207.23809325,      0.        , ...,
            0.        , 296301.18720493, 282436.80303509])

In [51]:
from sklearn.metrics import mean_squared_error as mse
mse(y_test,pred)

1721253986238915.8

In [52]:
y_test.head()

75721    0.0
80184    0.0
19864    0.0
76699    0.0
92991    0.0
Name: totals.transactionRevenue, dtype: float64

TODO:

Impute missing values
    Set revnue to 0 if not present
scale

 0   channelGrouping                               100000 non-null  object
* 1   date                                          100000 non-null  int64 
- 2   fullVisitorId                                 100000 non-null  object
- 3   sessionId                                     100000 non-null  object
 4   socialEngagementType                          100000 non-null  object
- 5   visitId                                       100000 non-null  int64 
 6   visitNumber                                   100000 non-null  int64 
* 7   visitStartTime                                100000 non-null  int64 
 8   device.browser                                100000 non-null  object
 9   device.operatingSystem                        100000 non-null  object
 10  device.isMobile                               100000 non-null  bool  
 11  device.deviceCategory                         100000 non-null  object
 12  geoNetwork.continent                          100000 non-null  object
 13  geoNetwork.subContinent                       100000 non-null  object
 14  geoNetwork.country                            100000 non-null  object
 15  geoNetwork.region                             100000 non-null  object
 16  geoNetwork.metro                              100000 non-null  object
 17  geoNetwork.city                               100000 non-null  object
* 18  geoNetwork.networkDomain                      100000 non-null  object
 19  totals.visits                                 100000 non-null  object
 20  totals.hits                                   100000 non-null  object
 21  totals.pageviews                              99993 non-null   object
 22  totals.bounces                                48916 non-null   object
 23  totals.newVisits                              77263 non-null   object
 24  totals.transactionRevenue                     1399 non-null    object
 25  trafficSource.campaign                        100000 non-null  object
 26  trafficSource.source                          100000 non-null  object
 27  trafficSource.medium                          100000 non-null  object
 28  trafficSource.keyword                         44218 non-null   object
 29  trafficSource.isTrueDirect                    30454 non-null   object
 30  trafficSource.referralPath                    36473 non-null   object
 31  trafficSource.adwordsClickInfo.page           2574 non-null    object
 32  trafficSource.adwordsClickInfo.slot           2574 non-null    object
 33  trafficSource.adwordsClickInfo.gclId          2625 non-null    object
 34  trafficSource.adwordsClickInfo.adNetworkType  2574 non-null    object
 35  trafficSource.adwordsClickInfo.isVideoAd      2574 non-null    object
 36  trafficSource.adContent                       1325 non-null    object
 37  trafficSource.campaignCode                    1 non-null       object

In [53]:
train["geoNetwork.networkDomain"].head()

0    6360
1    1762
2    6597
3    6597
4    6597
Name: geoNetwork.networkDomain, dtype: int64

In [54]:
payingCustomers["totals.transactionRevenue"].head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: totals.transactionRevenue, dtype: float64

In [55]:
payingCustomers["totals.transactionRevenue"].describe()

count    1.000000e+05
mean     1.819409e+06
std      3.901549e+07
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      5.498000e+09
Name: totals.transactionRevenue, dtype: float64

In [56]:
payingCustomers.head()

,channelGrouping,date,fullVisitorId,visitId,visitNumber,visitStartTime,device.browser,device.operatingSystem,device.isMobile,device.deviceCategory,geoNetwork.continent,geoNetwork.subContinent,geoNetwork.country,geoNetwork.region,geoNetwork.metro,geoNetwork.city,geoNetwork.networkDomain,totals.hits,totals.pageviews,totals.bounces,totals.newVisits,totals.transactionRevenue,trafficSource.campaign,trafficSource.source,trafficSource.medium,trafficSource.keyword,trafficSource.isTrueDirect,trafficSource.referralPath,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adContent
0,4,20160902,1131660440785968503,1472830385,1.0,1.472830e+09,12,17,False,0,3,21,182,129,0,189,10216,1.0,1.0,1.0,1.0,0.0,3,74,5,5,1,1386,6,3,8098,3,1,47
1,4,20160902,377306020877927890,1472880147,1.0,1.472880e+09,17,7,False,0,5,1,9,318,78,459,2805,1.0,1.0,1.0,1.0,0.0,3,74,5,5,1,1386,6,3,8098,3,1,47
2,4,20160902,3895546263509774583,1472865386,1.0,1.472865e+09,12,17,False,0,4,19,164,68,0,233,10632,1.0,1.0,1.0,1.0,0.0,3,74,5,5,1,1386,6,3,8098,3,1,47
3,4,20160902,4763447161404445595,1472881213,1.0,1.472881e+09,40,6,False,0,3,16,82,318,78,459,10632,1.0,1.0,1.0,1.0,0.0,3,74,5,285,1,1386,6,3,8098,3,1,47
4,4,20160902,27294437909732085,1472822600,2.0,1.472823e+09,12,1,True,1,4,13,189,318,78,459,10632,1.0,1.0,1.0,0.0,0.0,3,74,5,5,0,1386,6,3,8098,3,1,47


## Deployment

Suggestion for deployment: Write a web API which can receive data like one or more rows from the test dataset and return a prediction for that data. We are thinking the client will send the data as JSON and receive a JSON response.

In order to make this happen, we will need to have a way to transform our data to JSON format, as well as a way to transform it back to a dataframe. 

In [57]:
train.select_dtypes(exclude=["number","bool_","object_"])

""
0
1
2
3
4
...
99995
99996
99997
99998


In [58]:
train.select_dtypes(np.number).head()

,channelGrouping,date,visitId,visitNumber,visitStartTime,device.browser,device.operatingSystem,device.deviceCategory,geoNetwork.continent,geoNetwork.subContinent,geoNetwork.country,geoNetwork.region,geoNetwork.metro,geoNetwork.city,geoNetwork.networkDomain,totals.hits,totals.pageviews,totals.bounces,totals.newVisits,totals.transactionRevenue,trafficSource.campaign,trafficSource.source,trafficSource.medium,trafficSource.keyword,trafficSource.isTrueDirect,trafficSource.referralPath,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adContent
0,4,20160902,1472830385,1.0,1.472830e+09,5,12,0,3,21,168,93,0,118,6360,1.0,1.0,1.0,1.0,0.0,0,49,5,5,1,527,4,2,2389,2,1,21
1,4,20160902,1472880147,1.0,1.472880e+09,8,7,0,5,1,9,217,52,289,1762,1.0,1.0,1.0,1.0,0.0,0,49,5,5,1,527,4,2,2389,2,1,21
2,4,20160902,1472865386,1.0,1.472865e+09,5,12,0,4,19,151,49,0,145,6597,1.0,1.0,1.0,1.0,0.0,0,49,5,5,1,527,4,2,2389,2,1,21
3,4,20160902,1472881213,1.0,1.472881e+09,26,6,0,3,16,76,217,52,289,6597,1.0,1.0,1.0,1.0,0.0,0,49,5,203,1,527,4,2,2389,2,1,21
4,4,20160902,1472822600,2.0,1.472823e+09,5,1,1,4,13,174,217,52,289,6597,1.0,1.0,1.0,0.0,0.0,0,49,5,5,0,527,4,2,2389,2,1,21


In [59]:
train.select_dtypes(exclude=["number","bool_","object_"])

""
0
1
2
3
4
...
99995
99996
99997
99998


## Setting up the server

In [ ]:
import flask
from flask import request

app = flask.Flask(__name__)

@app.route('/', methods=['GET'])
def home():
    return "<h1>API for Google store revenue predictions</h1>"

@app.route('/', methods=['POST'])
def predict():
    data = request.get_json()
    data = prepare(pd.read_json(data)) # Prepares the data for a prediction
    pred = model.predict(data)         
    res = prepare_result(pred, data)   # Prepares the result by constructing a dataframe of {id, predicted_revenue}
    return app.response_class(
        response=res.to_json(),
        status=200,
        mimetype='application/json'
    )
    

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


### TODO/Checklist

* Create pipeline
    * Create transformers for:
        * Remove useless columns
        * Impute missing values
        * Transform total revenue to natural log of total revenue
        
        
* Test the model to make sure it performs well-ish
        
* Store model to a file using pickle or joblib
    
    

* Create API - The API will receive raw data in the same format as our initial CSV files. It will then have to:
    * Remove missing columns
    * impute missing values
    * any other preprocessing
    * Make prediction
    * Return a JSON object containing customer ID and transactionrevenue. 
* Test API
   
We might start by only accepting single row data and then expand to allow multiple row data. 

### If we have time:
* Display data in nice ways to gain insights
* Try different models
* Deploy API